In [2]:
from llama_cpp import Llama

#llm = Llama.from_pretrained(repo_id="NousResearch/Hermes-2-Pro-Llama-3-8B-GGUF", filename= "*Q4_K_M.gguf", 
                            #n_gpu_layers = -1,
                            #n_ctx=512,
                            #max_tokens=12,
                            #n_batch=512,
                            #n_threads=6,
                            #verbose=True,
                            #flash_attn=True)

Spin up server using `llama-cpp-python[server]` package.
```bash	
python -m llama_cpp.server --hf_model_repo_id NousResearch/Hermes-2-Pro-Llama-3-8B-GGUF --model '*Q4_K_M.gguf' --n_ctx 512 --n_batch 256 --n_threads 6 --flash_attn True --chat_format chatml --n_gpu_layers 30  
```

In [16]:
from openai import OpenAI


client = OpenAI(
    base_url="http://localhost:8000/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)

schema = {
    "type": "object",
    "properties": {
        "rating": {
            "type": "number",
            "minimum": 1,
            "maximum": 5,
            "description": "The rating of the joke, from 1 to 5.",
        }
    },
}

joke = "Why did the scarecrow win an award? Because he was outstanding in his field!"

def rate_joke(joke: str):
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": f"You are a joke evaluator that answers in JSON. Here's the json schema you must adhere to:\n<schema>\n{schema}\n</schema>",
        },
        {"role": "user",
            "content": f""" Your task is to evaluate jokes based on their funniness on a scale from 1 to 5, 
                            where 1 represents the least funny and 5 represents the most funny. 
                            Consider the humor, originality, and overall impact of the joke when making your assessment: \n "{joke}" """},
    ],
    #response_format= {"type": "json_object"}, #"schema": schema}, # uncomment this line to enforce the schema
    temperature=0.2,
    top_p=0.9,
    max_tokens=10,
    )

    return completion.choices[0].message.content

rate_joke(joke)

'{\n  "rating": 4\n}'

In [17]:
import pandas as pd
pretraining_data = pd.read_parquet('../data/processed/pretraining_data.parquet')

# take a subset of 100 jokes to test
pretraining_data = pretraining_data.sample(100)

pretraining_data['rating'] = pretraining_data['cleaned_joke'].apply(rate_joke)

In [20]:
pretraining_data['rating'].value_counts()

rating
3    57
2    29
4    11
1     3
Name: count, dtype: int64

In [19]:
# Compare the ratings to the original ratings {"rating": 3}  keep only the number
pretraining_data['rating'] = pretraining_data['rating'].str.extract('(\d+)')
pretraining_data['rating'] = pretraining_data['rating'].astype(int)

from sklearn.metrics import accuracy_score

accuracy_score(pretraining_data['rating'], pretraining_data['score_class'])


0.2